In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
from collections import Counter

In [ ]:
#mount your drive if using colab

In [ ]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform = transforms.Compose([transforms.Resize(256),
                                transforms.RandomResizedCrop(224),
                                transforms.RandomHorizontalFlip(),
                                transforms.RandomRotation(30),
                                transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.4),
                                transforms.ToTensor(),
                                transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
                                ])

In [ ]:
train_dataset = datasets.ImageFolder('./Training', transform=transform)
test_dataset = datasets.ImageFolder('.Testing', transform=transform)

In [ ]:
condition_names = ['Glioma', 'Meningioma', 'No Tumor', 'Pituitary']


class_counts = Counter([label for _, label in train_dataset.imgs])
total_samples = sum(class_counts.values())
class_weights = [total_samples / class_counts[i] for i in range(len(condition_names))]
class_weights = torch.tensor(class_weights, dtype=torch.float, device=device)

class_sample_counts = [class_counts[i] for i in range(len(condition_names))]
weights = 1. / torch.tensor(class_sample_counts, dtype=torch.float)
sample_weights = [weights[label] for _, label in train_dataset.imgs]

sampler = torch.utils.data.WeightedRandomSampler(sample_weights, num_samples=len(train_dataset), replacement=True)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, sampler=sampler)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
class CombinedModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.resnet = torchvision.models.resnet18(pretrained=True)
        for param in self.resnet.parameters():
            param.requires_grad = False

        for param in self.resnet.layer1.parameters():
            param.requires_grad = True
        for param in self.resnet.layer2.parameters():
            param.requires_grad = True
        for param in self.resnet.layer3.parameters():
            param.requires_grad = True
        for param in self.resnet.layer4.parameters():
            param.requires_grad = True

        for param in self.resnet.fc.parameters():
            param.requires_grad = True

        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Identity()

        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)      #outputs (32,224,224) before pooling by self.pool = nn.MaxPool2d(2,2)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(2,2)                                #outputs (32,112,112) after pooling by self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)     #outputs (64,56,56)   after pooling by self.pool = nn.MaxPool2d(2,2)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)         #outputs (128,28,28)  after pooling by self.pool = nn.MaxPool2d(2,2)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)       #outputs (256, 14,14) after pooling by self.pool = nn.MaxPool2d(2,2)
        self.bn4 = nn.BatchNorm2d(256)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(50688, 512)
        self.fc2 = nn.Linear(512, 4)


    def forward(self, x):
        resnet_features = self.resnet(x)

        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = self.dropout(x)

        x = torch.flatten(x, 1)
        combined_features = torch.cat((resnet_features, x), dim=1)
        x = F.relu(self.fc1(combined_features))
        x = self.fc2(x)

        return x

In [ ]:
combinedModel = CombinedModel()
combinedModel.to(device)


loss_function = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(combinedModel.parameters(), lr=1e-4)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [ ]:
train_losses = []
test_accuracies = []
precision_scores = []
recall_scores = []
f1_scores = []

for epoch in range(30):
    print(f"Training epoch {epoch+1} ...")

    combinedModel.train()
    running_loss = 0.0


    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()

        output = combinedModel(inputs)
        loss = loss_function(output, labels)

        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    train_losses.append(avg_loss)

    combinedModel.eval()
    correct = 0
    total = 0
    all_labels = []
    all_predictions = []


    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = combinedModel(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    accuracy = 100 * correct / total
    test_accuracies.append(accuracy)

    precision = precision_score(all_labels, all_predictions, average='weighted', zero_division=0)
    recall = recall_score(all_labels, all_predictions, average='weighted', zero_division=0)
    f1 = f1_score(all_labels, all_predictions, average='weighted')

    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)

    print(f"Epoch {epoch+1}/{30} - Loss: {avg_loss:.4f} - Accuracy: {accuracy:.2f}%")

    lr_scheduler.step()


fig, ax1 = plt.subplots()
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss', color='tab:blue')
ax1.plot(range(1, 31), train_losses, color='tab:red', marker='o', label='Train Loss')
ax1.tick_params(axis='y', labelcolor='tab:red')

ax2 = ax1.twinx()
ax2.set_ylabel('Accuracy (%)', color='tab:green')
ax2.plot(range(1, 31), test_accuracies, color='tab:blue', marker= 'o', label='Test Accuracy')
ax2.tick_params(axis='y', labelcolor='tab:blue')

plt.title('Training Loss and Test Accuracy Over Epochs')
plt.legend(loc='upper left')
plt.show()

# Confusion matrix
cm = confusion_matrix(all_labels, all_predictions)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=condition_names, yticklabels=condition_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

print(f"Precision: {precision:.4f} - Recall: {recall:.4f} - F1 Score: {f1:.4f}")

# Classification report
print("\nClassification Report:\n")
print(classification_report(all_labels, all_predictions, target_names=condition_names))